In [1]:
import numpy as np
import pandas as pd
import scipy

In [2]:
data_dir = '/s/project/mll/sergey/effect_prediction/MLM/griesemer/SVR/skew_pred/'

In [3]:
cells = ['HEK293FT', 'HMEC', 'HEPG2', 'GM12878', 'K562', 'SKNSH']
models = ['MLM','4mers','word2vec','griesemer','enformer_summary','enformer_all_targets']

In [4]:
#collect results for all cells and all models

N_rounds = 1000 #limit the number of train test splits

all_res = []

for cell_type in cells:
    for model in models:
        if not '5mers' in model:
            model_dir = data_dir + cell_type + '/' + model + '/'
            df = pd.read_csv(model_dir + 'cv_scores.tsv', sep='\t')
            df['model'] = model
            df['cell'] = cell_type
            df.rename(columns={'scores':'r2'}, inplace=True)
            all_res.append(df.iloc[:N_rounds])

all_res = pd.concat(all_res)

FileNotFoundError: [Errno 2] No such file or directory: '/s/project/mll/sergey/effect_prediction/MLM/griesemer/SVR/skew_pred/HEK293FT/enformer_all_targets/cv_scores.tsv'

In [ ]:
score_df = all_res[['cell','model','round','r2']].set_index(['cell','model']).sort_index()

In [24]:
#get p-value for each pair of models for each cell type
#use paired t-test with correction for repeated CV (C. Nadeau and Y. Bengio. Inference for the generalization error. In Machine Learning 52:239–281, 2003)

test_train_ratio = 0.11 #ratio between test and train counts in each split

sign_tests = {}

for cell in cells:
    cell_res = np.zeros((len(models),len(models)))
    for model1_idx,model1 in enumerate(models):
        for model2_idx,model2 in enumerate(models):
            if model1!=model2:
                score1 = score_df.loc[(cell, model1)].set_index('round').r2 #scores for model1
                score2 = score_df.loc[(cell, model2)].set_index('round').r2 #scores for model2
                diff_score = score1-score2
                t_stat = diff_score.mean()/np.sqrt((1/N_rounds + test_train_ratio)*diff_score.var())
                pval = scipy.stats.t.sf(np.abs(t_stat), N_rounds-1)*2  # two-sided pvalue
            else:
                pval = np.NaN
            cell_res[model1_idx,model2_idx] = pval
    sign_tests[cell] = pd.DataFrame(cell_res, columns=models,index=models)
    
sign_tests = pd.concat(sign_tests)

In [25]:
sign_tests.applymap(lambda x:'-' if np.isnan(x) else f'{x:.3f}' if x>1e-3 else '<1e-3')

MLM  4mers word2vec griesemer enformer_summary
HEK293FT MLM                   -  0.501    0.300     0.232            0.322
         4mers             0.501      -    0.417     0.533            0.556
         word2vec          0.300  0.417        -     0.943            0.818
         griesemer         0.232  0.533    0.943         -            0.769
         enformer_summary  0.322  0.556    0.818     0.769                -
HMEC     MLM                   -  0.549    0.487     0.831            0.604
         4mers             0.549      -    0.876     0.672            0.950
         word2vec          0.487  0.876        -     0.733            0.953
         griesemer         0.831  0.672    0.733         -            0.722
         enformer_summary  0.604  0.950    0.953     0.722                -
HEPG2    MLM                   -  0.589    0.549     0.820            0.971
         4mers             0.589      -    0.804     0.372            0.759
         word2vec          0.549  0.804        -     0.294            0.723
         griesemer         0.820  0.372    0.294         -            0.916
         enformer_summary  0.971  0.759    0.723     0.916                -
GM12878  MLM                   -  0.368    0.207     0.853            0.323
         4mers             0.368      -    0.978     0.185            0.995
         word2vec          0.207  0.978        -     0.169            0.977
         griesemer         0.853  0.185    0.169         -            0.153
         enformer_summary  0.323  0.995    0.977     0.153                -
K562     MLM                   -  0.424    0.561     0.446            0.764
         4mers             0.424      -    0.962     0.754            0.397
         word2vec          0.561  0.962        -     0.698            0.538
         griesemer         0.446  0.754    0.698         -            0.432
         enformer_summary  0.764  0.397    0.538     0.432                -
SKNSH    MLM                   -  0.409    0.607     0.259            0.901
         4mers             0.409      -    0.935     0.418            0.925
         word2vec          0.607  0.935        -     0.416            0.911
         griesemer         0.259  0.418    0.416         -            0.586
         enformer_summary  0.901  0.925    0.911     0.586                -

In [26]:
res_mean = [] #mean r2 for each model
res_std = [] #std for each model

for cell in cells:
    cell_df = score_df.loc[cell].r2.reset_index()
    res_mean.append(cell_df.groupby('model').mean().rename(columns={'r2':cell}))
    res_std.append(cell_df.groupby('model').std().rename(columns={'r2':cell}))

res_mean = pd.concat(res_mean,axis=1)
res_std = pd.concat(res_std,axis=1)

In [27]:
n_pairs = len(cells)*(len(models)-1) #for each cell the MLM model is compared with all other models
significance_thr = 0.05/n_pairs #bonferroni correction

def highlight_ns(x):
    #make the best model and models with insignificant difference with the best model bold
    best_model = 'MLM'#x.apply(lambda x:float(x.split('±')[0])).idxmax()
    cell = x.name
    sign_results = sign_tests.loc[(cell,best_model)]
    equal_models = list(sign_results[sign_results>significance_thr].index) #models with similar performance
    return ['font-weight: bold' if model==best_model or model in equal_models else ''
                for model in x.index]

res_all = res_mean.applymap(lambda x:f'{x:.4f}') + '±' + res_std.applymap(lambda x:f'{x:.4f}')

res_all.loc[models].style.apply(highlight_ns)

,HEK293FT,HMEC,HEPG2,GM12878,K562,SKNSH
model,,,,,,
MLM,-0.0003±0.0045,0.0031±0.0173,-0.0020±0.0093,0.0034±0.0151,-0.0015±0.0021,-0.0024±0.0045
4mers,-0.0011±0.0031,0.0003±0.0088,-0.0004±0.0032,-0.0009±0.0038,-0.0029±0.0051,-0.0015±0.0031
word2vec,-0.0017±0.0036,0.0007±0.0117,-0.0002±0.0031,-0.0008±0.0118,-0.0028±0.0066,-0.0013±0.0075
griesemer,-0.0018±0.0038,0.0020±0.0139,-0.0013±0.0035,0.0042±0.0119,-0.0038±0.0092,0.0009±0.0095
enformer_summary,-0.0015±0.0024,0.0005±0.0058,-0.0018±0.0139,-0.0009±0.0071,-0.0015±0.0020,-0.0019±0.0121


In [13]:
#bets hyperparameters

best_hpp = all_res[['cell','model','C','gamma','epsilon']].drop_duplicates().sort_values(by=['cell','model'])

best_hpp

,cell,model,C,gamma,epsilon
0,GM12878,4mers,64.330355,0.000201,0.368149
0,GM12878,MLM,5.312592,0.004606,0.376697
0,GM12878,griesemer,7.063151,0.004238,0.429876
0,GM12878,word2vec,3.471536,0.003538,0.392607
0,HEK293FT,4mers,1.667570,0.007338,0.038946
0,HEK293FT,MLM,17.941839,0.000576,0.308112
0,HEK293FT,griesemer,3.660187,0.002543,0.250479
0,HEK293FT,word2vec,32.343636,0.000811,0.177635
0,HEPG2,4mers,73.922564,0.000188,0.358625
0,HEPG2,MLM,5.066060,0.005798,0.357137
